In [1]:
import gc
import os
import pickle
import random
import shutil

import numpy as np
import pandas as pd
import tensorflow as tf
from keras import losses, optimizers, metrics, callbacks, layers, Model
from SpeechModels import get_melspec_model

In [2]:
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [3]:
SEED = 123
N_CLASS = 12
MAX_EPOCHS = 200

In [4]:
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

## Load data

In [5]:
train_ds = tf.keras.utils.audio_dataset_from_directory(
    directory="data/train",
    batch_size=512,
    output_sequence_length=16000,
    shuffle=True,
    seed=SEED
)

val_ds = tf.keras.utils.audio_dataset_from_directory(
    directory="data/val",
    batch_size=512,
    output_sequence_length=16000,
    shuffle=False
)

test_ds = tf.keras.utils.audio_dataset_from_directory(
    directory="data/test",
    batch_size=512,
    output_sequence_length=16000,
    shuffle=False
)

label_names = np.array(train_ds.class_names)
print("label names:", label_names)

Found 45586 files belonging to 12 classes.
Found 6513 files belonging to 12 classes.
Found 13024 files belonging to 12 classes.
label names: ['down' 'go' 'left' 'no' 'off' 'on' 'right' 'silence' 'stop' 'unknown'
 'up' 'yes']


In [6]:
def squeeze(audio, labels):
    audio = tf.squeeze(audio, axis=-1)
    return audio, labels

train_ds = train_ds.map(squeeze, tf.data.AUTOTUNE)
val_ds = val_ds.map(squeeze, tf.data.AUTOTUNE)
test_ds = test_ds.map(squeeze, tf.data.AUTOTUNE)

## Model stacked bidirectional lstm

In [7]:
def create_bidirectional_model():
    mel_spec_model = get_melspec_model(16000)
    inputs, outputs = mel_spec_model.inputs, mel_spec_model.outputs
    x = layers.Bidirectional(layers.LSTM(256, return_sequences=True))(outputs)
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=False))(x)
    out = layers.Dense(N_CLASS, activation="softmax")(x)
    return Model(inputs=inputs, outputs=out)

model = create_bidirectional_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 16000)]      0           []                               
                                                                                                  
 tf.signal.stft (TFOpLambda)    (None, 125, 513)     0           ['input[0][0]']                  
                                                                                                  
 tf.math.abs (TFOpLambda)       (None, 125, 513)     0           ['tf.signal.stft[0][0]']         
                                                                                                  
 tf.tensordot (TFOpLambda)      (None, 125, 80)      0           ['tf.math.abs[0][0]']            
                                                                                              

In [8]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss=losses.SparseCategoricalCrossentropy(),
    metrics=[metrics.SparseCategoricalAccuracy(), metrics.SparseCategoricalCrossentropy()]
)

early_stopping = callbacks.EarlyStopping(
    monitor='val_sparse_categorical_accuracy',
    min_delta=0,
    patience=5,
    verbose=0,
    mode='max',
    baseline=None,
    restore_best_weights=True
)

reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_sparse_categorical_accuracy', factor=0.5, patience=3, min_lr=0.00001, verbose=1)

In [9]:
history = model.fit(
    train_ds,
    epochs=2,
    validation_data=val_ds,
    shuffle=True,
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/2
90/90 [==============================] - 26s 152ms/step - loss: 1.3027 - sparse_categorical_accuracy: 0.6450 - sparse_categorical_crossentropy: 1.3027 - val_loss: 0.9185 - val_sparse_categorical_accuracy: 0.7198 - val_sparse_categorical_crossentropy: 0.9185 - lr: 0.0010
Epoch 2/2
90/90 [==============================] - 13s 135ms/step - loss: 0.7187 - sparse_categorical_accuracy: 0.7758 - sparse_categorical_crossentropy: 0.7187 - val_loss: 0.6293 - val_sparse_categorical_accuracy: 0.7979 - val_sparse_categorical_crossentropy: 0.6293 - lr: 0.0010


## Experiments

Training will be repeated 5 times with different weights initialization.

In [10]:
EXPERIMENT_NAME = "stacked_bidirectional_lstm"
if os.path.exists(EXPERIMENT_NAME):
    shutil.rmtree(EXPERIMENT_NAME)
    os.mkdir(EXPERIMENT_NAME)
else:
    os.mkdir(EXPERIMENT_NAME)

In [11]:
TRAINING_SEEDS = list(range(5))
results = []
for seed in TRAINING_SEEDS:
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    model = create_bidirectional_model()

    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.01),
        loss=losses.SparseCategoricalCrossentropy(),
        metrics=[metrics.SparseCategoricalAccuracy(), metrics.SparseCategoricalCrossentropy()]
    )

    history = model.fit(
        train_ds,
        epochs=MAX_EPOCHS,
        validation_data=val_ds,
        shuffle=True,
        callbacks=[early_stopping, reduce_lr]
    )

    with open(os.path.join(EXPERIMENT_NAME, f"history_{seed}.pkl"), "wb") as file:
        pickle.dump(history.history, file)

    eval_results = model.evaluate(test_ds)

    predictions = model.predict(test_ds)
    with open(os.path.join(EXPERIMENT_NAME, f"predictions_{seed}.pkl"), "wb") as file:
        pickle.dump(predictions, file)

    results += [{
        'seed': seed,
        'results': dict(zip(model.metrics_names, eval_results))
    }]
    gc.collect()

results = pd.DataFrame(results)
results = pd.concat([results.drop(["results"], axis=1), results["results"].apply(pd.Series)], axis=1)
results.to_csv(os.path.join(EXPERIMENT_NAME, 'results.csv'))

Epoch 1/200
90/90 [==============================] - 22s 150ms/step - loss: 1.5379 - sparse_categorical_accuracy: 0.6231 - sparse_categorical_crossentropy: 1.5379 - val_loss: 1.5102 - val_sparse_categorical_accuracy: 0.6320 - val_sparse_categorical_crossentropy: 1.5102 - lr: 0.0100
Epoch 2/200
90/90 [==============================] - 13s 133ms/step - loss: 1.4331 - sparse_categorical_accuracy: 0.6328 - sparse_categorical_crossentropy: 1.4331 - val_loss: 1.3785 - val_sparse_categorical_accuracy: 0.6326 - val_sparse_categorical_crossentropy: 1.3785 - lr: 0.0100
Epoch 3/200
90/90 [==============================] - 13s 132ms/step - loss: 1.1886 - sparse_categorical_accuracy: 0.6504 - sparse_categorical_crossentropy: 1.1886 - val_loss: 1.0311 - val_sparse_categorical_accuracy: 0.6803 - val_sparse_categorical_crossentropy: 1.0311 - lr: 0.0100
Epoch 4/200
90/90 [==============================] - 13s 132ms/step - loss: 0.8341 - sparse_categorical_accuracy: 0.7421 - sparse_categorical_crossentr

In [12]:
results

,seed,loss,sparse_categorical_accuracy,sparse_categorical_crossentropy
0,0,0.174891,0.949094,0.174891
1,1,0.181845,0.953087,0.181845
2,2,0.212424,0.944487,0.212424
3,3,0.169613,0.955390,0.169613
4,4,0.231978,0.942568,0.231978


## Experiments

Training will be repeated 5 times with different weights initialization.

In [13]:
def create_onedirectional_model():
    mel_spec_model = get_melspec_model(16000)
    inputs, outputs = mel_spec_model.inputs, mel_spec_model.outputs
    x = layers.LSTM(256, return_sequences=True)(outputs)
    x = layers.LSTM(128, return_sequences=True)(x)
    x = layers.LSTM(64, return_sequences=False)(x)
    out = layers.Dense(N_CLASS, activation="softmax")(x)
    return Model(inputs=inputs, outputs=out)

model = create_onedirectional_model()
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 16000)]      0           []                               
                                                                                                  
 tf.signal.stft_6 (TFOpLambda)  (None, 125, 513)     0           ['input[0][0]']                  
                                                                                                  
 tf.math.abs_6 (TFOpLambda)     (None, 125, 513)     0           ['tf.signal.stft_6[0][0]']       
                                                                                                  
 tf.tensordot_6 (TFOpLambda)    (None, 125, 80)      0           ['tf.math.abs_6[0][0]']          
                                                                                            

In [14]:
EXPERIMENT_NAME = "stacked_onedirectional_lstm"
if os.path.exists(EXPERIMENT_NAME):
    shutil.rmtree(EXPERIMENT_NAME)
    os.mkdir(EXPERIMENT_NAME)
else:
    os.mkdir(EXPERIMENT_NAME)

In [15]:
TRAINING_SEEDS = list(range(5))
results = []
for seed in TRAINING_SEEDS:
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    model = create_onedirectional_model()

    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.01),
        loss=losses.SparseCategoricalCrossentropy(),
        metrics=[metrics.SparseCategoricalAccuracy(), metrics.SparseCategoricalCrossentropy()]
    )

    history = model.fit(
        train_ds,
        epochs=MAX_EPOCHS,
        validation_data=val_ds,
        shuffle=True,
        callbacks=[early_stopping, reduce_lr]
    )

    with open(os.path.join(EXPERIMENT_NAME, f"history_{seed}.pkl"), "wb") as file:
        pickle.dump(history.history, file)

    eval_results = model.evaluate(test_ds)

    predictions = model.predict(test_ds)
    with open(os.path.join(EXPERIMENT_NAME, f"predictions_{seed}.pkl"), "wb") as file:
        pickle.dump(predictions, file)

    results += [{
        'seed': seed,
        'results': dict(zip(model.metrics_names, eval_results))
    }]
    gc.collect()

results = pd.DataFrame(results)
results = pd.concat([results.drop(["results"], axis=1), results["results"].apply(pd.Series)], axis=1)
results.to_csv(os.path.join(EXPERIMENT_NAME, 'results.csv'))

Epoch 1/200
90/90 [==============================] - 19s 163ms/step - loss: 1.5162 - sparse_categorical_accuracy: 0.6239 - sparse_categorical_crossentropy: 1.5162 - val_loss: 1.4140 - val_sparse_categorical_accuracy: 0.6318 - val_sparse_categorical_crossentropy: 1.4140 - lr: 0.0100
Epoch 2/200
90/90 [==============================] - 15s 158ms/step - loss: 1.3150 - sparse_categorical_accuracy: 0.6376 - sparse_categorical_crossentropy: 1.3150 - val_loss: 1.2414 - val_sparse_categorical_accuracy: 0.6429 - val_sparse_categorical_crossentropy: 1.2414 - lr: 0.0100
Epoch 3/200
90/90 [==============================] - 16s 161ms/step - loss: 1.0775 - sparse_categorical_accuracy: 0.6828 - sparse_categorical_crossentropy: 1.0775 - val_loss: 1.0878 - val_sparse_categorical_accuracy: 0.6751 - val_sparse_categorical_crossentropy: 1.0878 - lr: 0.0100
Epoch 4/200
90/90 [==============================] - 15s 160ms/step - loss: 0.8090 - sparse_categorical_accuracy: 0.7462 - sparse_categorical_crossentr

In [16]:
results

,seed,loss,sparse_categorical_accuracy,sparse_categorical_crossentropy
0,0,0.198255,0.952933,0.198255
1,1,0.250526,0.946944,0.250526
2,2,0.183877,0.953547,0.183877
3,3,0.195825,0.953317,0.195825
4,4,0.211138,0.949862,0.211138
